# Simulating the Kibble-Zurek Mechanism on a Quantum Computer

This notebook demonstrates the use of a quantum computer for ... . It has the following sections:

1. [The Kibble-Zurek Mechanism](#The-Kibble-Zurek-Mechanism) describes.
2. 

## The Kibble-Zurek Mechanism

The [[1]](#1) Nature paper

<img src="_static/temp1_kz_overview.png" width="500">


## References

<a name="1">[1]</a> King, A.D., Suzuki, S., Raymond, J. et al. 
Coherent quantum annealing in a programmable 2,000 qubit Ising chain. 
Nat. Phys. 18, 1324–1328 (2022). https://doi.org/10.1038/s41567-022-01741-6


Copyright &copy; 2024 D-Wave Systems, Inc

The software is licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This Jupyter Notebook is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>